# NGFC Modelling with IDEAS - pse
This is a modelling example of a NGFC system using idaes-pse framework.

In [1]:
from pyomo.environ import ConcreteModel, Constraint, Objective, SolverFactory, TransformationFactory, Constraint, Var
from pyomo.network import Arc

In [2]:
from idaes.core import FlowsheetBlock
from idaes.unit_models import Mixer, HeatExchanger, Separator, GibbsReactor, Heater
# Methane combustion ideal package got CH4, H2O, CO, CO2, N2, NH3, O2
import idaes.property_models.activity_coeff_models.methane_combustion_ideal as thermo_props
#import idaes.property_models.activity_coeff_models.methane_combustion_ideal as reaction_props

In [3]:
import matplotlib.pyplot as plt

In [4]:
from idaes.unit_models.separator import SplittingType
from idaes.unit_models.heat_exchanger import delta_temperature_amtd_callback
from idaes.core.util.model_statistics import degrees_of_freedom as dof

### Building Base Flowsheet.

In [5]:
m = ConcreteModel()
m.fs = FlowsheetBlock(default={"dynamic": False})
m.fs.thermo_params = thermo_props.MethaneParameterBlock()

In [6]:
# Fuel ultilization (Uf): mole reductant consumed in FC per mole of reductant total
Uf = 0.8
# Air ultilization (Ua): mole of air consumed in FC per mole of air feed
Ua = 0.2
# Methane to steam ratio (MS): mole methane per mole water
MS = 2
# Feed:
# Reaction: 
# Reforming: CH4 + H2O -> CO + 3H2
# Water gas shift: CO + H2O -> CO2 + H2
# Methane combustion: CH4 + 2O2 -> CO2 + 2H2O
# Hydrogen combustion: H2 + 1/2O2 -> H2O
# Carbon monoxide combustion: CO + 1/2O2 -> CO2

n_CH4f = 10
print("mole of methane feed: "+str(n_CH4f)+" mole/s")
n_H2Of = n_CH4f*MS
print("mole of steam feed: "+str(n_H2Of)+" mole/s")
n_O2f = n_CH4f*Uf*2/Ua
n_N2f = n_O2f*0.79/0.21
print("mole of air feed: "+str(n_N2f+n_O2f)+" mole/s")

n_H2ex = 2
n_COex = n_CH4f*(1-Uf)*4-n_H2ex
n_CO2ex = n_CH4f-n_COex
n_H2Oex = n_H2Of+2*n_CH4f-n_H2ex
y_H2ex = n_H2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_COex = n_COex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_CO2ex = n_CO2ex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)
y_H2Oex = n_H2Oex/(n_H2ex + n_COex + n_CO2ex + n_H2Oex)

print("Anode exhaust: ")
print("y_H2ex: "+str(y_H2ex))
print("y_COex: "+str(y_COex))
print("y_CO2ex: "+str(y_CO2ex))
print("y_H2Oex: "+str(y_H2Oex))
print("Total mole/s: "+str(n_H2ex + n_COex + n_CO2ex + n_H2Oex))

n_N2ex = n_N2f
n_O2ex = n_O2f - n_CH4f*Uf*2
y_O2ex = n_O2ex/(n_O2ex+n_N2ex)
y_N2ex = n_N2ex/(n_O2ex+n_N2ex)
print("Cathode exhaust: ")
print("y_O2ex: "+str(y_O2ex))
print("y_N2ex: "+str(y_N2ex))
print("Total mole/s: "+str(n_O2ex+n_N2ex))

mole of methane feed: 10 mole/s
mole of steam feed: 20 mole/s
mole of air feed: 380.95238095238096 mole/s
Anode exhaust: 
y_H2ex: 0.04
y_COex: 0.11999999999999997
y_CO2ex: 0.08000000000000003
y_H2Oex: 0.76
Total mole/s: 50.0
Cathode exhaust: 
y_O2ex: 0.1753653444676409
y_N2ex: 0.824634655532359
Total mole/s: 364.95238095238096


In [7]:
# Temperature User input
T_FC_air_in = 700 + 273.15
T_FC_fuel_in = 400 + 273.15
T_FC_ex_out = 800 + 273.15

### Declare all Units:

In [8]:
m.fs.HX1 = HeatExchanger(default={"dynamic": False,
                                  "delta_temperature_callback": delta_temperature_amtd_callback,
                                  "shell":{"property_package": m.fs.thermo_params},
                                  "tube":{"property_package": m.fs.thermo_params}})

In [9]:
m.fs.HX2 = HeatExchanger(default={"dynamic": False,
                                  "delta_temperature_callback": delta_temperature_amtd_callback,
                                  "shell":{"property_package": m.fs.thermo_params},
                                  "tube":{"property_package": m.fs.thermo_params}})

In [10]:
m.fs.HX3 = HeatExchanger(default={"dynamic": False,
                                  "delta_temperature_callback": delta_temperature_amtd_callback,
                                  "shell":{"property_package": m.fs.thermo_params},
                                  "tube":{"property_package": m.fs.thermo_params}})

In [11]:
m.fs.Heater = Heater(default={"property_package": m.fs.thermo_params})

In [12]:
m.fs.Mix1 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [13]:
m.fs.Mix2 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [14]:
m.fs.Mix3 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [15]:
m.fs.Mix4 = Mixer(default={"dynamic": False,
                           "num_inlets": 3,
                           "property_package": m.fs.thermo_params})

In [16]:
m.fs.Split1 = Separator(default={"dynamic": False,
                                 "split_basis": SplittingType.componentFlow,
                                 "property_package": m.fs.thermo_params})

In [17]:
m.fs.Split2 = Separator(default={"dynamic": False,
                                 "num_outlets": 3,
                                 "split_basis": SplittingType.totalFlow,
                                 "property_package": m.fs.thermo_params})

In [18]:
m.fs.Reformer = GibbsReactor(default={"dynamic": False,
                                      "property_package": m.fs.thermo_params,
                                      "has_pressure_change": False,
                                      "has_heat_transfer": True})

In [19]:
m.fs.SOFC = GibbsReactor(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params,
                                  "has_pressure_change": False,
                                  "has_heat_transfer": True})

In [20]:
m.fs.Burner = GibbsReactor(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params,
                                    "has_pressure_change": False,
                                    "has_heat_transfer": True})

### Declare all Streams:

In [21]:
m.fs.stream0 = Arc(source=m.fs.Mix1.outlet,
                   destination=m.fs.HX1.tube_inlet)

In [22]:
m.fs.stream1 = Arc(source=m.fs.Split1.outlet_1,
                   destination=m.fs.HX3.tube_inlet)

In [23]:
m.fs.stream2 = Arc(source=m.fs.HX1.tube_outlet,
                   destination=m.fs.Reformer.inlet)

In [24]:
m.fs.stream3 = Arc(source=m.fs.Split1.outlet_2,
                   destination=m.fs.HX2.tube_inlet)

In [25]:
m.fs.stream4 = Arc(source=m.fs.Reformer.outlet,
                   destination=m.fs.Mix2.inlet_1)

In [26]:
m.fs.stream5 = Arc(source=m.fs.HX3.tube_outlet,
                   destination=m.fs.Mix2.inlet_2)

In [27]:
m.fs.stream6 = Arc(source=m.fs.Mix2.outlet,
                   destination=m.fs.SOFC.inlet)

In [28]:
m.fs.stream7 = Arc(source=m.fs.HX2.tube_outlet,
                   destination=m.fs.Mix3.inlet_2)

In [29]:
m.fs.stream8 = Arc(source=m.fs.SOFC.outlet,
                   destination=m.fs.Mix3.inlet_1)

In [30]:
m.fs.stream9 = Arc(source=m.fs.Mix3.outlet,
                   destination=m.fs.Burner.inlet)

In [31]:
m.fs.stream10 = Arc(source=m.fs.Burner.outlet,
                    destination=m.fs.Split2.inlet)

In [32]:
m.fs.stream11 = Arc(source=m.fs.Split2.outlet_1,
                    destination=m.fs.HX1.shell_inlet)

In [33]:
m.fs.stream12 = Arc(source=m.fs.Split2.outlet_2,
                    destination=m.fs.HX2.shell_inlet)

In [34]:
m.fs.stream13 = Arc(source=m.fs.Split2.outlet_3,
                    destination=m.fs.HX3.shell_inlet)

In [35]:
m.fs.stream14 = Arc(source=m.fs.HX1.shell_outlet,
                    destination=m.fs.Mix4.inlet_1)

In [36]:
m.fs.stream150 = Arc(source=m.fs.HX2.shell_outlet,
                     destination=m.fs.Heater.inlet)

In [37]:
m.fs.stream15 = Arc(source=m.fs.Heater.outlet,
                    destination=m.fs.Mix4.inlet_2)

In [38]:
m.fs.stream16 = Arc(source=m.fs.HX3.shell_outlet,
                    destination=m.fs.Mix4.inlet_3)

In [39]:
TransformationFactory("network.expand_arcs").apply_to(m)

### Define known Material Streams:

In [40]:
# Fix methane flow to Mix1:
m.fs.Mix1.inlet_1.flow_mol.fix(n_CH4f)
m.fs.Mix1.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.Mix1.inlet_1.mole_frac_comp[0.0,"CH4"].fix(1.0)
m.fs.Mix1.inlet_1.temperature.fix(25+273.15)
m.fs.Mix1.inlet_1.pressure.fix(101325)

In [41]:
# Fix water flow to Mix1:
m.fs.Mix1.inlet_2.flow_mol.fix(n_H2Of)
m.fs.Mix1.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.Mix1.inlet_2.mole_frac_comp[0.0,"H2O"].fix(1.0)
m.fs.Mix1.inlet_2.temperature.fix(25+273.15)
m.fs.Mix1.inlet_2.pressure.fix(101325)

In [42]:
# Fix air flow to Split1:
m.fs.Split1.inlet.flow_mol.fix(n_N2f+n_O2f)
m.fs.Split1.inlet.mole_frac_comp[0.0,:].fix(0.0)
m.fs.Split1.inlet.mole_frac_comp[0.0,"O2"].fix(0.21)
m.fs.Split1.inlet.mole_frac_comp[0.0,"N2"].fix(0.79)
m.fs.Split1.inlet.temperature.fix(25+273.15)
m.fs.Split1.inlet.pressure.fix(101325)

In [43]:
# Fix O2 flow in Split1 outlet_1:
m.fs.Split1.outlet_1.flow_mol.fix(n_CH4f*Uf*2)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"CH4"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"CO"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"CO2"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"H2"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"H2O"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"N2"].fix(0.0)
m.fs.Split1.outlet_1.mole_frac_comp[0.0,"O2"].fix(1.0)

### Constraints:  Adiabatic Burner

In [44]:
m.fs.Burner.heat_duty.fix(0.0)

In [45]:
m.fs.Reformer.heat_duty.fix(0.0)

### Constraints: Reformer temperature

In [46]:
m.fs.Reformer.outlet.temperature.fix(T_FC_fuel_in)
print(T_FC_fuel_in-273.15)

400.0


### Constraints: SOFC exit temperature

In [47]:
m.fs.SOFC.outlet.temperature.fix(T_FC_ex_out)
print(T_FC_ex_out-273.15)

800.0000000000001


### Constraints: Heat Exchanger tube outlet temperature

In [48]:
m.fs.HX2.tube_outlet.temperature.fix(T_FC_ex_out)
print(T_FC_ex_out-273.15)

800.0000000000001


In [49]:
m.fs.HX3.tube_outlet.temperature.fix(T_FC_air_in)
print(T_FC_air_in-273.15)

700.0


### Constraints: Heat duty in Heater

In [50]:
m.fs.Heater.heat_duty.fix(-n_H2Of*43988)

In [51]:
m.fs.Heater.heat_duty.display()

heat_duty : Size=1, Index=fs.time
    Key : Lower : Value   : Upper : Fixed : Stale : Domain
    0.0 :  None : -879760 :  None :  True : False :  Reals


### Constraints: Heat Exchangers 

In [52]:
m.fs.HX1.area.fix(500)

In [53]:
m.fs.HX2.area.fix(500)

In [54]:
m.fs.HX3.area.fix(500)

In [55]:
m.fs.HX1.shell_outlet.temperature.fix(120+273.15)

In [56]:
m.fs.HX3.shell_outlet.temperature.fix(120+273.15)

In [ ]:
m.fs.Split2.split_fraction[0.0, "outlet_1"].fix(0.2)

In [ ]:
m.fs.Split2.split_fraction.display()

In [ ]:
m.fs.Split2.split_fraction[0.0, "outlet_2"].fix(0.8)

In [ ]:
m.fs.HX1.overall_heat_transfer_coefficient.fix()

### Constraints: HX1 delta temperature in (T_stream11 - T_stream2)

In [ ]:
m.fs.HX1.delta_temperature_in.fix(30)

In [ ]:
m.fs.HX1.delta_temperature_out.fix(250)

### Constraints: HX2 delta temperature in (T_stream15 - T_stream3)

In [ ]:
m.fs.HX2.delta_temperature_out.fix(20)

### Constraints: HX3 delta temperature

In [ ]:
m.fs.HX3.delta_temperature_out.fix(60)

### Constraints: Split2 first fraction

In [ ]:
m.fs.SOFC.heat_duty.fix(6040000)

In [ ]:
dof(m.fs.HX2)

In [ ]:
m.fs.Split2.split_fraction.display()

In [ ]:
m.fs.Mix1.outlet.display()

In [57]:
dof(m)

0

## Initialize Stuff

In [58]:
m.fs.Mix1.initialize()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-05-18 22:16:42 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.mixed_state State Released.
2020-05-18 22:16:42 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.inlet_1_state State Released.
2020-05-18 22:16:42 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.inlet_2_state State Released.


In [59]:
m.fs.Mix2.initialize()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-05-18 22:17:02 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix2.mixed_state State Released.
2020-05-18 22:17:02 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix2.inlet_1_state State Released.
2020-05-18 22:17:02 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix2.inlet_2_state State Released.


In [60]:
m.fs.Mix3.initialize()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-05-18 22:17:45 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix3.mixed_state State Released.
2020-05-18 22:17:45 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix3.inlet_1_state State Released.
2020-05-18 22:17:45 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix3.inlet_2_state State Released.


In [61]:
m.fs.Mix4.initialize()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-05-18 22:17:56 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.mixed_state State Released.
2020-05-18 22:17:56 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.inlet_1_state State Released.
2020-05-18 22:17:56 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.inlet_2_state State Released.
2020-05-18 22:17:56 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.inlet_3_state State Released.


In [62]:
m.fs.Split1.initialize()

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-05-18 22:18:22 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Split1.outlet_1_state State Released.


KeyError: 'fs.Split1 state variable and split fraction indexing sets do not match. The in-built  initialization routine for Separators relies on the split fraction and state variable indexing sets matching to calculate initial guesses for extensive variables. In other cases users will need to provide their own initial guesses'

In [58]:
m.fs.Mix1.initialize()
m.fs.Mix2.initialize()
m.fs.Mix4.initialize()
m.fs.Split2.initialize()
m.fs.HX1.initialize()
m.fs.HX2.initialize()
m.fs.HX3.initialize()
m.fs.Heater.initialize()


--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-04-28 10:42:55 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.mixed_state State Released.
2020-04-28 10:42:55 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.inlet_1_state State Released.
2020-04-28 10:42:55 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix1.inlet_2_state State Released.


--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

...

--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-04-28 10:42:57 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.mixed_state State Released.
2020-04-28 10:42:57 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.inlet_1_state State Released.
2020-04-28 10:42:57 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.inlet_2_state State Released.
2020-04-28 10:42:58 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Mix4.inlet_3_state State Released.


--- Logging error ---
Traceback (most recent call last):
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 1025, in emit
    msg = self.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 869, in format
    return fmt.format(record)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 608, in format
    record.message = record.getMessage()
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/logging/__init__.py", line 369, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/adlerlabadmin/anaconda3/envs/idaes-env/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/adlerlabadmin/a

2020-04-28 10:42:58 - INFO - idaes.property_models.activity_coeff_models.activity_coeff_prop_pack - fs.Split1.outlet_1_state State Released.


KeyError: 'fs.Split1 state variable and split fraction indexing sets do not match. The in-built  initialization routine for Separators relies on the split fraction and state variable indexing sets matching to calculate initial guesses for extensive variables. In other cases users will need to provide their own initial guesses'

In [ ]:
m.fs.Split2.split_fraction.display()

In [59]:
solver = SolverFactory('ipopt')

In [60]:
results = solver.solve(m, tee=True)

Ipopt 3.13.2: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.13.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:     3991
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1067

Total number of variables............................:     1154
                     variables with only lower bounds:      107
                variables with lower and upper bounds:      609
                     variables with only upper bounds:        0
Total

 189r 0.0000000e+00 2.15e+03 2.04e+05  -4.6 6.40e+04    -  4.94e-01 3.68e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 2.10e+03 1.98e+05  -4.6 4.56e+04    -  1.62e-01 2.57e-02f  1
 191r 0.0000000e+00 2.09e+03 1.97e+05  -4.6 4.47e+04    -  1.50e-02 5.96e-03f  1
 192r 0.0000000e+00 2.08e+03 1.96e+05  -4.6 4.44e+04    -  2.85e-02 3.96e-03f  1
 193r 0.0000000e+00 2.07e+03 1.95e+05  -4.6 4.43e+04    -  1.49e-03 5.78e-03f  1
 194r 0.0000000e+00 2.07e+03 1.95e+05  -4.6 4.41e+04    -  1.25e-01 1.29e-06f  1
 195r 0.0000000e+00 1.91e+03 1.81e+05  -4.6 4.41e+04    -  1.00e+00 7.69e-02f  1
 196r 0.0000000e+00 1.68e+03 1.58e+05  -4.6 4.12e+04    -  8.50e-02 1.30e-01f  1
 197r 0.0000000e+00 1.05e+03 4.62e+04  -4.6 3.67e+04    -  9.35e-07 8.37e-01f  1
 198r 0.0000000e+00 3.98e+03 8.92e+04  -4.6 2.17e+04    -  5.30e-02 4.22e-01f  1
 199r 0.0000000e+00 3.98e+03 2.77e+04  -4.6 6.85e+03    -  8.54e-01 7.89e-04f  1
iter    objective    inf_pr 

ValueError: Cannot load a SolverResults object with bad status: error

In [ ]:
print(results)

In [ ]:
m.fs.Reformer.display()

In [ ]:
m.fs.HX2.display()

In [ ]:
m.fs.Mix4.inlet_1.display()

In [ ]:
m.fs.Mix4.inlet_2.display()

In [ ]:
m.fs.Mix4.inlet_3.display()

In [ ]:
m.fs.Burner.outlet.display()

In [ ]:
m.fs.SOFC.heat_duty.display()

In [ ]:
m.fs.Split2.split_fraction.display()

In [ ]:
m.fs.HX1.inlet_1.display()

In [ ]:
m.fs.HX1.tube_inlet.display()

In [ ]:
m.fs.Mix1.inlet_1.display()

In [ ]:
m.fs.Mix1.inlet_2.display()

In [ ]:
m.fs.Mix1.outlet.display()

In [ ]:
m.fs.Heater.display()

In [ ]:
m.fs.Mix4.inlet_1.display()

In [ ]:
354.9024586155912-273.15

### First mixer (Mix1) combines methane and water stream:

In [ ]:
m.fs.mix1 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [ ]:
m.fs.mix1.inlet_1.flow_mol.fix(n_CH4f)
m.fs.mix1.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix1.inlet_1.mole_frac_comp[0.0,"CH4"].fix(1.0)
m.fs.mix1.inlet_1.temperature.fix(25+273.15)
m.fs.mix1.inlet_1.pressure.fix(101325)

In [ ]:
m.fs.mix1.inlet_2.flow_mol.fix(n_H2Of)
m.fs.mix1.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix1.inlet_2.mole_frac_comp[0.0,"H2O"].fix(1.0)
m.fs.mix1.inlet_2.temperature.fix(25+273.15)
m.fs.mix1.inlet_2.pressure.fix(101325)

In [ ]:
m.fs.mix1.inlet_1.display()

In [ ]:
m.fs.mix1.inlet_2.display()

### First heat exchanger (HX1) heats up methane & water stream
### Also reformer (Reformer) is a Gibbs Reactor that receive heat from HX1 exhaust stream

In [ ]:
# Set up reformer?
m.fs.reformer = GibbsReactor(default={"dynamic": False,
                                      "property_package": m.fs.thermo_params,
                                      "has_pressure_change": False,
                                      "has_heat_transfer": True})

In [ ]:
m.fs.stream1 = Arc(source=m.fs.mix1.outlet,
                   destination=m.fs.reformer.inlet)

In [ ]:
m.fs.reformer.outlet.temperature.fix(T_FC_fuel_in+273.15)

### HX1: Provide the heat duty for Reformer

### Seperator (Sep1) split the air into a single of stream of O2 going to SOFC

### Mix2: combined reformer outlet stream and O2 stream from cathode.

In [ ]:
m.fs.mix2 = Mixer(default={"dynamic": False,
                           "property_package": m.fs.thermo_params})

In [ ]:
m.fs.stream2 = Arc(source=m.fs.reformer.outlet,
                   destination=m.fs.mix2.inlet_1)

In [ ]:
m.fs.mix2.inlet_2.flow_mol.fix(n_CH4f*Uf*2)
m.fs.mix2.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix2.inlet_2.mole_frac_comp[0.0,"O2"].fix(1.0)
m.fs.mix2.inlet_2.temperature.fix(T_FC_air_in+273.15)
m.fs.mix2.inlet_2.pressure.fix(101325)

### SOFC: Gibbs Reactor w stream coming from mix2

In [ ]:
# Set up SOFC
m.fs.SOFC = GibbsReactor(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params,
                                  "has_pressure_change": False,
                                  "has_heat_transfer": True})

In [ ]:
m.fs.stream3 = Arc(source=m.fs.mix2.outlet,
                   destination=m.fs.SOFC.inlet)

In [ ]:
m.fs.SOFC.outlet.temperature.fix(T_FC_ex_out+273.15)

In [ ]:
m.fs.mix1.initialize()

In [ ]:
m.fs.mix2.initialize()

### SOFC outlet come into the burner with the rest of the air
### Burner: Gibbs Reactor

In [ ]:
# Feed to the burner

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
m.fs.reformer.inlet.display()

In [ ]:
m.fs.reformer.outlet.display()

In [ ]:
m.fs.reformer.heat_duty.display()

In [ ]:
m.fs.mix2.inlet_1.display()

In [ ]:
m.fs.mix2.inlet_2.display()

In [ ]:
m.fs.mix2.outlet.display()

In [ ]:
m.fs.SOFC.inlet.display()

In [ ]:
m.fs.SOFC.outlet.display()

In [ ]:
m.fs.SOFC.heat_duty.display()

In [ ]:
m.fs.burner = GibbsReactor(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params,
                                    "has_pressure_change": False,
                                    "has_heat_transfer": True})

### SOFC: Gibbs Reactor, taking Reformer outlet and a specified O2 amount from Mix2

### Mixing anode & cathode exhaust:

In [ ]:
m.fs.mix_exhaust = Mixer(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params})

In [ ]:
# First inlet to mix_exhaust : anode exhaust
m.fs.mix_exhaust.inlet_1.flow_mol.fix(50.0)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"H2"].fix(0.04)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"CO"].fix(0.12)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"CO2"].fix(0.08)
m.fs.mix_exhaust.inlet_1.mole_frac_comp[0.0,"H2O"].fix(0.76)
m.fs.mix_exhaust.inlet_1.temperature.fix(T_FC_ex_out+273.15)
m.fs.mix_exhaust.inlet_1.pressure.fix(101325)

In [ ]:
m.fs.mix_exhaust.inlet_1.display()

In [ ]:
# Second inlet to mix_exhaust : cathode exhaust
m.fs.mix_exhaust.inlet_2.flow_mol.fix(364.95238095238096)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,:].fix(0.0)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,"O2"].fix(0.1753653444676409)
m.fs.mix_exhaust.inlet_2.mole_frac_comp[0.0,"N2"].fix(0.824634655532359)
m.fs.mix_exhaust.inlet_2.temperature.fix(T_FC_ex_out+273.15)
m.fs.mix_exhaust.inlet_2.pressure.fix(101325)

In [ ]:
m.fs.mix_exhaust.inlet_2.display()

### Burner as Gibbs Reactor

In [ ]:
m.fs.burner = GibbsReactor(default={"dynamic": False,
                                    "property_package": m.fs.thermo_params,
                                    "has_pressure_change": False,
                                    "has_heat_transfer": True})

### Point mixer stream to Burner:

In [ ]:
m.fs.stream1 = Arc(source=m.fs.mix_exhaust.outlet,
                  destination=m.fs.burner.inlet)

In [ ]:
TransformationFactory("network.expand_arcs").apply_to(m)

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m)

In [ ]:
m.fs.burner.heat_duty.fix(0.0)

In [ ]:
from idaes.core.util.model_statistics import degrees_of_freedom as dof
dof(m)

In [ ]:
solver = SolverFactory('ipopt')

In [ ]:
results = solver.solve(m, tee=True)

In [ ]:
print(results)

In [ ]:
m.fs.burner.inlet.display()

In [ ]:
m.fs.burner.outlet.display()

### Make another Gibbs Reactor to represent the SOFC

In [ ]:
m.fs.SOFC = GibbsReactor(default={"dynamic": False,
                                  "property_package": m.fs.thermo_params,
                                  "has_pressure_change": False,
                                  "has_heat_transfer": True})